In [1]:
import sys
# adding llms to the system path, make sure to configure the file azureopenai in llms folder with end point details
sys.path.append('C:\\proddev\\research\\aiagentssamples\\llms\\')
import azureopenai
import autogen
from autogen import GroupChat, GroupChatManager
from autogen import AssistantAgent
from autogen import UserProxyAgent


llm_config = {
    "seed": 44,  # for caching the response
    "config_list": azureopenai.config_list,
    "temperature": 0  # for coding it is suggested to be less creative
}


financial_tasks = [
    """What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?""",
    """Investigate possible reasons of the stock performance.""",
    """Plot a graph comparing the stock prices over the past month.""",
]

writing_tasks = ["""Develop an engaging blog post using any information provided."""]

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)

financial_assistant = autogen.AssistantAgent(
    name="Financial_assistant",
    llm_config=llm_config,
)

research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
)

writer = autogen.AssistantAgent(
    name="Writer",
    llm_config={"config_list": azureopenai.config_list},
    system_message="""
    You are a professional writer, known for
    your insightful and engaging articles.
    You transform complex concepts into compelling narratives.
    Reply "TERMINATE" in the end when everything is done.
    """,
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="""Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.
    Reply "TERMINATE" in the end when everything is done.
    """,
    llm_config={"config_list": azureopenai.config_list},
)

groupchat_1 = autogen.GroupChat(agents=[user_proxy, research_assistant, critic], messages=[], max_round=50)

groupchat_2 = autogen.GroupChat(agents=[user_proxy, writer, critic], messages=[], max_round=50)

manager_1 = autogen.GroupChatManager(
    groupchat=groupchat_1,
    name="Research_manager",
    llm_config={"config_list": azureopenai.config_list},
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat_mgr_1",
        "use_docker": False,
    },
)
manager_2 = autogen.GroupChatManager(
    groupchat=groupchat_2,
    name="Writing_manager",
    llm_config={"config_list": azureopenai.config_list},
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat_mgr_2",
        "use_docker": False,
    },
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "grouptasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)
await user.a_initiate_chats(  # noqa: F704
    [
        {"chat_id": 1, "recipient": financial_assistant, "message": financial_tasks[0], "summary_method": "last_msg"},
        {
            "chat_id": 2,
            "prerequisites": [1],
            "recipient": manager_1,
            "message": financial_tasks[1],
            "summary_method": "reflection_with_llm",
        },
        {
            "chat_id": 3,
            "prerequisites": [1],
            "recipient": manager_1,
            "message": financial_tasks[2],
            "summary_method": "reflection_with_llm",
        },
        {"chat_id": 4, "prerequisites": [1, 2, 3], "recipient": manager_2, "message": writing_tasks[0]},
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
User (to Financial_assistant):

What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

--------------------------------------------------------------------------------
Financial_assistant (to User):

To get the current stock prices and the performance over the past month for NVDA (NVIDIA) and TSLA (Tesla), we can use a Python script that utilizes a financial data API. One such API is the Yahoo Finance API, which can be accessed using the `yfinance` library in Python.

Here is the plan:
1. Install the `yfinance` library if it's not already installed.
2. Fetch the current stock prices for NVDA and TSLA.
3. Fetch the stock prices from one month ago.
4. Calculate the percentage change over the past month.
5. Print the results.

L

c:\proddev\research\.venv\Lib\site-packages\autogen\agentchat\chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


User (to Financial_assistant):

exitcode: 0 (execution succeeded)
Code output: 
Ticker: NVDA
Current Price: 138.07
Price One Month Ago: 116.78
Percentage Change Over the Past Month: 18.23%

Ticker: TSLA
Current Price: 219.16
Price One Month Ago: 226.78
Percentage Change Over the Past Month: -3.36%



--------------------------------------------------------------------------------
Financial_assistant (to User):

Based on the output of the script, here are the current stock prices and the performance over the past month for NVDA and TSLA:

- **NVIDIA (NVDA)**
  - Current Price: $138.07
  - Price One Month Ago: $116.78
  - Percentage Change Over the Past Month: +18.23%

- **Tesla (TSLA)**
  - Current Price: $219.16
  - Price One Month Ago: $226.78
  - Percentage Change Over the Past Month: -3.36%

This indicates that NVIDIA's stock price has increased by 18.23% over the past month, while Tesla's stock price has decreased by 3.36% over the same period.

TERMINATE

-------------------------

{1: ChatResult(chat_id=1, chat_history=[{'content': 'What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?', 'role': 'assistant', 'name': 'User'}, {'content': 'To get the current stock prices and the performance over the past month for NVDA (NVIDIA) and TSLA (Tesla), we can use a Python script that utilizes a financial data API. One such API is the Yahoo Finance API, which can be accessed using the `yfinance` library in Python.\n\nHere is the plan:\n1. Install the `yfinance` library if it\'s not already installed.\n2. Fetch the current stock prices for NVDA and TSLA.\n3. Fetch the stock prices from one month ago.\n4. Calculate the percentage change over the past month.\n5. Print the results.\n\nLet\'s write the Python script to achieve this:\n\n```python\n# filename: stock_prices.py\n\nimport yfinance as yf\nfrom datetime import datetime, timedelta\n\n# Define the stock tickers\ntickers = [\'NVDA\', \'TSLA\']\